# Problem

In [59]:
function generate_distance_matrix(n; random_seed = seed)
    rng = Random.MersenneTwister(random_seed)
    X = 100 * rand(rng, n) # x 좌표
    Y = 100 * rand(rng, n) # y 좌표
    d = [sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2) for i in 1:n, j in 1:n] #distance
    return X, Y, d
end

generate_distance_matrix (generic function with 2 methods)

# Algorithm

In [60]:
using Random

function tsp_dynamic_programming(distance_matrix)
    n = size(distance_matrix, 1)
    num_states = 2^n
    dp = fill(Inf, n, num_states)

    # Initialize the base case
    dp[1, 1] = 0

    # Calculate the minimum distance for each state
    for mask in 1:num_states
        for u in 2:n
            if (mask & (1 << (u - 1))) != 0
                for v in 1:n
                    ###### Fill in the code here ######
                    
                end
            end
        end
    end


    # Calculate the minimum tour length
    tour_length = Inf
    for u in 2:n
        println("Current u: ", u)
        print("Current tour_length: ", round(tour_length, digits=4))
        ###### Fill in the code here ######
        
    end

    return tour_length
end


tsp_dynamic_programming (generic function with 1 method)

# Experiment

In [69]:
using Concorde

function run_tsp_dp(n; seed = 1234)
    X, Y, d = generate_distance_matrix(n; random_seed = seed)
    tour_length = tsp_dynamic_programming(d)
    println("----------------------------------------------------")
    println("Minimum tour length for $n cities: ", tour_length)
end


run_tsp_dp (generic function with 1 method)

In [70]:
run_tsp_dp(10)

Current u: 2
Current tour_length: Inf vs 302.3404
Current u: 3
Current tour_length: 302.3404 vs 314.7005
Current u: 4
Current tour_length: 302.3404 vs 285.4396
Current u: 5
Current tour_length: 285.4396 vs 271.9402
Current u: 6
Current tour_length: 271.9402 vs 287.5675
Current u: 7
Current tour_length: 271.9402 vs 305.6292
Current u: 8
Current tour_length: 271.9402 vs 289.4072
Current u: 9
Current tour_length: 271.9402 vs 330.336
Current u: 10
Current tour_length: 271.9402 vs 271.9402
----------------------------------------------------
Minimum tour length for 10 cities: 271.94015912296027


In [71]:
run_tsp_dp(10; seed=0)

Current u: 2
Current tour_length: Inf vs 289.3781
Current u: 3
Current tour_length: 289.3781 vs 307.5528
Current u: 4
Current tour_length: 289.3781 vs 334.9734
Current u: 5
Current tour_length: 289.3781 vs 289.3781
Current u: 6
Current tour_length: 289.3781 vs 318.6574
Current u: 7
Current tour_length: 289.3781 vs 330.2518
Current u: 8
Current tour_length: 289.3781 vs 333.7976
Current u: 9
Current tour_length: 289.3781 vs 303.2786
Current u: 10
Current tour_length: 289.3781 vs 293.8271
----------------------------------------------------
Minimum tour length for 10 cities: 289.3780898449664
